# EDA

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import sys
import xml.etree.ElementTree as ET
import pandas as pd
import re
import string
import os
import numpy as np
import spacy

## Entrenamiento

### 2019

In [ ]:
data_list = []

In [ ]:
directory = 'T2/train/2019/data'
# Recorrer todos los archivos en el directorio
for path in os.listdir(directory):
    filename = os.path.join(directory, path)
    try:
        tree = ET.parse(filename)
        root = tree.getroot()
        user = root.find('ID').text
        writings = root.findall('WRITING')
        text = ' '.join((writing.find('TEXT').text).strip() for writing in writings)
        new_data= [user, text]
        data_list.append(new_data)
    except Exception as e:
        print(f"Error processing file {filename}: {e}")

In [ ]:
train1 = pd.DataFrame(data_list, columns=['user', 'text'])

In [ ]:
train1['text'] = train1['text'].fillna('')

In [ ]:
file = open('T2/train/2019/T1_erisk_golden_truth.txt', 'r')
for line in file:
    usuario = line.split()[0]
    clasificacion = int(line.split(' ')[1])
    train1.loc[train1['user'] == usuario, 'label'] = int(clasificacion)

In [ ]:
train1.to_csv('train1_TFG.csv', index=False)

### 2018

In [ ]:
data_list = []
processed_users =[]

In [ ]:
directory = 'T2/train/2018/test'
# Recorrer todos los archivos en el directorio
for path in os.listdir(directory):
    filename = os.path.join(directory, path)
    try:
        tree = ET.parse(filename)
        root = tree.getroot()
        user = root.find('ID').text
        writings = root.findall('WRITING')
        text = ' '.join((writing.find('TEXT').text).strip() for writing in writings)
        if user in processed_users:
            index = processed_users.index(user)
            data_list[index][1] += text
        else:
            new_data= [user, text]
            data_list.append(new_data)
            processed_users.append(user)
    except Exception as e:
        print(f"Error processing file {filename}: {e}")

In [ ]:
train2 = pd.DataFrame(data_list, columns=['user', 'text'])

In [ ]:
train2['text'] = train2['text'].fillna('')

In [ ]:
file = open('T2/train/2018/test/risk-golden-truth-test.txt', 'r')
for line in file:
    usuario = line.split()[0]
    clasificacion = int(line.split(' ')[1])
    train2.loc[train2['user'] == usuario, 'label'] = int(clasificacion)

In [ ]:
train2.to_csv('train2_TFG.csv', index=False)

### Concatenar 2018 y 2019

In [ ]:
train1 = pd.read_csv('train1_TFG.csv')
train2 = pd.read_csv('train2_TFG.csv')
#concatenar los dos dataframes
train = pd.concat([train1, train2])
train.to_csv('train_TFG.csv', index=False)

## Prueba

Vamos a utilizar los datos de entrenamiento de 2018.

In [ ]:
data_list = []
processed_users =[]

In [ ]:
directory1 = 'T2/train/2018/train/negative_examples'
directory2 = 'T2/train/2018/train/positive_examples'
# Recorrer todos los archivos en el directorio
for directory in [directory1, directory2]:
    for path in os.listdir(directory):
        filename = os.path.join(directory, path)
        try:
            tree = ET.parse(filename)
            root = tree.getroot()
            user = root.find('ID').text
            writings = root.findall('WRITING')
            text = ' '.join((writing.find('TEXT').text).strip() for writing in writings)
            if user in processed_users:
                index = processed_users.index(user)
                data_list[index][1].join(text)
            else:
                new_data= [user, text]
                data_list.append(new_data)
                processed_users.append(user)
        except Exception as e:
            print(f"Error processing file {filename}: {e}")

In [ ]:
test = pd.DataFrame(data_list, columns=['user', 'text'])

In [ ]:
test['text'] = test['text'].fillna('')

In [ ]:
file = open('T2/train/2018/train/risk_golden_truth.txt', 'r')
for line in file:
    usuario = line.split()[0]
    clasificacion = int(line.split(' ')[1])
    test.loc[test['user'] == usuario, 'label'] = int(clasificacion)

In [ ]:
test.to_csv('test_TFG.csv', index=False)

### All data

In [ ]:
train = pd.read_csv('train_TFG.csv')
test = pd.read_csv('test_TFG.csv')
final = pd.concat([train, test])
final.to_csv('data_TFG.csv', index=False)

# STATS TFG 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Carga los datos
data = pd.read_csv('data_TFG.csv')


In [ ]:
# Añade la columna de longitud
data['length_text'] = data['text'].apply(lambda x: len(str(x).split()))

In [ ]:
# Configura el tamaño del gráfico
fig_size = (20, 10)
plt.figure(figsize=fig_size)

#eleminar filas cuya lenght sea 0
data = data[data['length_text'] != 0]
# Define las etiquetas personalizadas
etiquetas_personalizadas = {0: 'No anorexia', 1: 'Anorexia'}

# Obtiene las etiquetas únicas
labels = sorted(data["label"].unique())

# Crea un gráfico para cada etiqueta
for name in labels:
    # Selecciona los datos para esta etiqueta
    subset = data[data['label'] == name]
    # Dibuja el gráfico de densidad de Kernel
    sns.kdeplot(subset['length_text'], label=etiquetas_personalizadas[name], legend=True)

# Configura los ejes y la leyenda
plt.xlabel('Número de tokens del texto original', fontsize=14 )
plt.ylabel('Densidad', fontsize=14)
plt.legend(prop={'size': 15}, title='Etiquetas', loc="upper right")

# Muestra el gráfico
plt.show()


In [ ]:
# Obtiene un resumen estadístico de la columna 'length_text'
resumen = data['length_text'].describe(percentiles=[0, 0.25, 0.50, 0.75, 0.95])

print(resumen)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Carga los datos
data = pd.read_csv('data.csv')


In [ ]:
data['length_text'] = data['text'].apply(lambda x: len(str(x).split()))

In [ ]:
# Configura el tamaño del gráfico
fig_size = (20, 10)
plt.figure(figsize=fig_size)

# Define las etiquetas personalizadas
etiquetas_personalizadas = {0: 'No anorexia', 1: 'Anorexia'}

# Obtiene las etiquetas únicas
labels = sorted(data["label"].unique())

# Crea un gráfico para cada etiqueta
for name in labels:
    # Selecciona los datos para esta etiqueta
    subset = data[data['label'] == name]
    # Dibuja el gráfico de densidad de Kernel
    sns.kdeplot(subset['length_text'], label=etiquetas_personalizadas[name], legend=True)

# Configura los ejes y la leyenda
plt.xlabel('Número de tokens del texto original', fontsize=14 )
plt.ylabel('Densidad', fontsize=14)
plt.legend(prop={'size': 15}, title='Etiquetas', loc="upper right")

# Muestra el gráfico
plt.show()


In [ ]:
# Obtiene un resumen estadístico de la columna 'length_text'
resumen = data['length_text'].describe(percentiles=[0, 0.25, 0.50, 0.75, 0.95])

print(resumen)
